<a href="https://colab.research.google.com/github/666git/Chess/blob/master/%E4%B8%AD%E5%9B%BD%E8%B1%A1%E6%A3%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tkinter as tk
from tkinter import messagebox
import os

class ChineseChess:
    def __init__(self, root):
        self.root = root
        self.root.title("中国象棋")
        self.root.resizable(False, False)

        # 棋盘尺寸
        self.BOARD_WIDTH = 9  # 9条横线
        self.BOARD_HEIGHT = 10  # 10条竖线
        self.CELL_SIZE = 60  # 每个格子的大小
        self.PIECE_RADIUS = 25  # 棋子半径

        # 布局设置
        self.canvas_width = (self.BOARD_WIDTH - 1) * self.CELL_SIZE  # 修改这里
        self.canvas_height = (self.BOARD_HEIGHT - 1) * self.CELL_SIZE  # 修改这里
        self.margin = 40  # 边缘留白

        # 创建画布
        self.canvas = tk.Canvas(root, width=self.canvas_width + 2 * self.margin,
                                height=self.canvas_height + 2 * self.margin, bg="#F5DEB3")
        self.canvas.pack(padx=10, pady=10)

        # 游戏状态
        self.selected_piece = None
        self.current_player = 'red'  # 红方先行
        self.game_over = False

        # 初始化棋盘
        self.board = [[None for _ in range(self.BOARD_WIDTH)] for _ in range(self.BOARD_HEIGHT)]

        # 绘制棋盘
        self.draw_board()

        # 初始化棋子
        self.init_pieces()

        # 绑定点击事件
        self.canvas.bind("<Button-1>", self.handle_click)

        # 添加状态显示
        self.status_var = tk.StringVar()
        self.status_var.set("红方回合")
        status_label = tk.Label(root, textvariable=self.status_var, font=("Arial", 14))
        status_label.pack(pady=5)

        # 添加重新开始按钮
        restart_button = tk.Button(root, text="重新开始", command=self.restart_game)
        restart_button.pack(pady=5)

    def draw_board(self):
        """绘制棋盘"""
        # 绘制底板
        self.canvas.create_rectangle(
            self.margin, self.margin,
            self.canvas_width + self.margin, self.canvas_height + self.margin,
            fill="#F5DEB3", outline="#000000", width=2
        )

        # 绘制横线
        for i in range(self.BOARD_HEIGHT):
            y = i * self.CELL_SIZE + self.margin
            self.canvas.create_line(
                self.margin, y,
                (self.BOARD_WIDTH - 1) * self.CELL_SIZE + self.margin, y,  # 修改这里
                fill="#000000", width=2 if i == 0 or i == self.BOARD_HEIGHT - 1 else 1
            )

        # 绘制竖线
        for i in range(self.BOARD_WIDTH):
            x = i * self.CELL_SIZE + self.margin
            # 上半部分
            self.canvas.create_line(
                x, self.margin,
                x, 4 * self.CELL_SIZE + self.margin,
                fill="#000000", width=2 if i == 0 or i == self.BOARD_WIDTH - 1 else 1
            )
            # 下半部分
            self.canvas.create_line(
                x, 5 * self.CELL_SIZE + self.margin,
                x, (self.BOARD_HEIGHT - 1) * self.CELL_SIZE + self.margin,  # 修改这里
                fill="#000000", width=2 if i == 0 or i == self.BOARD_WIDTH - 1 else 1
            )

        # 绘制中间的楚河汉界
        self.canvas.create_rectangle(
            self.margin, 4 * self.CELL_SIZE + self.margin,
            (self.BOARD_WIDTH - 1) * self.CELL_SIZE + self.margin, 5 * self.CELL_SIZE + self.margin,  # 修改这里
            fill="#F5DEB3", outline="#000000"
        )
        self.canvas.create_text(
            self.canvas_width // 4 + self.margin, 4.5 * self.CELL_SIZE + self.margin,
            text="楚河", font=("Arial", 20), fill="#000000"
        )
        self.canvas.create_text(
            3 * self.canvas_width // 4 + self.margin, 4.5 * self.CELL_SIZE + self.margin,
            text="汉界", font=("Arial", 20), fill="#000000"
        )

        # 绘制九宫格
        # 上方九宫格
        self.canvas.create_line(
            3 * self.CELL_SIZE + self.margin, self.margin,
            5 * self.CELL_SIZE + self.margin, 2 * self.CELL_SIZE + self.margin,
            fill="#000000"
        )
        self.canvas.create_line(
            5 * self.CELL_SIZE + self.margin, self.margin,
            3 * self.CELL_SIZE + self.margin, 2 * self.CELL_SIZE + self.margin,
            fill="#000000"
        )

        # 下方九宫格
        self.canvas.create_line(
            3 * self.CELL_SIZE + self.margin, 7 * self.CELL_SIZE + self.margin,
            5 * self.CELL_SIZE + self.margin, 9 * self.CELL_SIZE + self.margin,
            fill="#000000"
        )
        self.canvas.create_line(
            5 * self.CELL_SIZE + self.margin, 7 * self.CELL_SIZE + self.margin,
            3 * self.CELL_SIZE + self.margin, 9 * self.CELL_SIZE + self.margin,
            fill="#000000"
        )

        # 绘制点位
        point_positions = [
            (1, 2), (7, 2),  # 炮位
            (1, 7), (7, 7),
            (0, 3), (2, 3), (4, 3), (6, 3), (8, 3),  # 兵位
            (0, 6), (2, 6), (4, 6), (6, 6), (8, 6)   # 卒位
        ]

        for col, row in point_positions:
            x = col * self.CELL_SIZE + self.margin
            y = row * self.CELL_SIZE + self.margin
            self.draw_point(x, y)

    def draw_point(self, x, y):
        """绘制棋盘上的点位标记"""
        point_size = 6
        for dx, dy in [(-1, -1), (-1, 1), (1, -1), (1, 1)]:
            # 检查是否在棋盘边缘
            in_x_edge = x <= self.margin or x >= (self.BOARD_WIDTH - 1) * self.CELL_SIZE + self.margin  # 修改这里
            in_y_edge = y <= self.margin or y >= (self.BOARD_HEIGHT - 1) * self.CELL_SIZE + self.margin  # 修改这里

            if (in_x_edge and dx < 0) or (in_y_edge and dy < 0):
                continue

            self.canvas.create_line(
                x, y,
                x + dx * point_size, y,
                fill="#000000", width=2
            )
            self.canvas.create_line(
                x, y,
                x, y + dy * point_size,
                fill="#000000", width=2
            )

    def init_pieces(self):
        """初始化棋子"""
        # 红方棋子（下方）
        self.create_piece(0, 9, "车", "red")
        self.create_piece(1, 9, "马", "red")
        self.create_piece(2, 9, "相", "red")
        self.create_piece(3, 9, "仕", "red")
        self.create_piece(4, 9, "帅", "red")
        self.create_piece(5, 9, "仕", "red")
        self.create_piece(6, 9, "相", "red")
        self.create_piece(7, 9, "马", "red")
        self.create_piece(8, 9, "车", "red")
        self.create_piece(1, 7, "炮", "red")
        self.create_piece(7, 7, "炮", "red")
        self.create_piece(0, 6, "兵", "red")
        self.create_piece(2, 6, "兵", "red")
        self.create_piece(4, 6, "兵", "red")
        self.create_piece(6, 6, "兵", "red")
        self.create_piece(8, 6, "兵", "red")

        # 黑方棋子（上方）
        self.create_piece(0, 0, "车", "black")
        self.create_piece(1, 0, "马", "black")
        self.create_piece(2, 0, "象", "black")
        self.create_piece(3, 0, "士", "black")
        self.create_piece(4, 0, "将", "black")
        self.create_piece(5, 0, "士", "black")
        self.create_piece(6, 0, "象", "black")
        self.create_piece(7, 0, "马", "black")
        self.create_piece(8, 0, "车", "black")
        self.create_piece(1, 2, "炮", "black")
        self.create_piece(7, 2, "炮", "black")
        self.create_piece(0, 3, "卒", "black")
        self.create_piece(2, 3, "卒", "black")
        self.create_piece(4, 3, "卒", "black")
        self.create_piece(6, 3, "卒", "black")
        self.create_piece(8, 3, "卒", "black")

    def create_piece(self, col, row, name, color):
        """创建棋子"""
        # 棋子应该放在格子的交叉点上
        x = col * self.CELL_SIZE + self.margin
        y = row * self.CELL_SIZE + self.margin

        # 创建棋子外圈
        circle = self.canvas.create_oval(
            x - self.PIECE_RADIUS, y - self.PIECE_RADIUS,
            x + self.PIECE_RADIUS, y + self.PIECE_RADIUS,
            fill="#f0d9b5" if color == "red" else "#b58863",
            outline="#000000", width=2
        )

        # 创建棋子文字
        text_color = "#FF0000" if color == "red" else "#000000"
        text = self.canvas.create_text(
            x, y, text=name, font=("Arial", 18, "bold"),
            fill=text_color
        )

        # 保存棋子信息
        piece = {
            'col': col,
            'row': row,
            'name': name,
            'color': color,
            'circle': circle,
            'text': text
        }

        # 更新棋盘数据
        self.board[row][col] = piece

    def handle_click(self, event):
        """处理鼠标点击事件"""
        if self.game_over:
            return

        # 将点击位置转换为棋盘坐标
        col = int((event.x - self.margin + self.CELL_SIZE/2) / self.CELL_SIZE)
        row = int((event.y - self.margin + self.CELL_SIZE/2) / self.CELL_SIZE)

        # 检查坐标是否在棋盘范围内
        if not (0 <= col < self.BOARD_WIDTH and 0 <= row < self.BOARD_HEIGHT):
            return

        # 如果点击了棋子
        if self.board[row][col] is not None:
            piece = self.board[row][col]

            # 如果点击了自己的棋子，则选中它
            if piece['color'] == self.current_player:
                self.select_piece(row, col)
                return

        # 如果已选中棋子，尝试移动
        if self.selected_piece is not None:
            self.move_piece(row, col)

    def select_piece(self, row, col):
        """选中棋子"""
        # 取消之前的选择
        if self.selected_piece is not None:
            selected_row, selected_col = self.selected_piece
            piece = self.board[selected_row][selected_col]
            self.canvas.itemconfig(
                piece['circle'],
                outline="#000000",
                width=2
            )

        # 选中新棋子
        piece = self.board[row][col]
        self.canvas.itemconfig(
            piece['circle'],
            outline="#00FF00",
            width=3
        )
        self.selected_piece = (row, col)

    def move_piece(self, target_row, target_col):
        """移动棋子"""
        selected_row, selected_col = self.selected_piece
        piece = self.board[selected_row][selected_col]

        # 检查移动是否合法
        if not self.is_valid_move(selected_row, selected_col, target_row, target_col):
            return

        # 目标位置的棋子
        target_piece = self.board[target_row][target_col]

        # 如果目标位置有敌方棋子，移除它
        if target_piece is not None:
            self.canvas.delete(target_piece['circle'])
            self.canvas.delete(target_piece['text'])

            # 判断是否吃掉将帅，结束游戏
            if target_piece['name'] in ["将", "帅"]:
                winner = "红方" if self.current_player == "red" else "黑方"
                messagebox.showinfo("游戏结束", f"{winner}胜利！")
                self.game_over = True

        # 更新棋盘数据
        self.board[target_row][target_col] = piece
        self.board[selected_row][selected_col] = None

        # 更新棋子位置
        x1 = selected_col * self.CELL_SIZE + self.margin
        y1 = selected_row * self.CELL_SIZE + self.margin
        x2 = target_col * self.CELL_SIZE + self.margin
        y2 = target_row * self.CELL_SIZE + self.margin

        # 移动棋子图形
        self.canvas.coords(
            piece['circle'],
            x2 - self.PIECE_RADIUS, y2 - self.PIECE_RADIUS,
            x2 + self.PIECE_RADIUS, y2 + self.PIECE_RADIUS
        )
        self.canvas.coords(piece['text'], x2, y2)

        # 更新棋子数据
        piece['col'] = target_col
        piece['row'] = target_row

        # 取消选中状态
        self.canvas.itemconfig(piece['circle'], outline="#000000", width=2)
        self.selected_piece = None

        # 切换玩家
        self.current_player = "black" if self.current_player == "red" else "red"
        self.status_var.set(f"{'红方' if self.current_player == 'red' else '黑方'}回合")

    def is_valid_move(self, from_row, from_col, to_row, to_col):
        """检查移动是否合法"""
        piece = self.board[from_row][from_col]
        target = self.board[to_row][to_col]

        # 不能移动到自己的棋子上
        if target is not None and target['color'] == piece['color']:
            return False

        # 根据不同的棋子类型检查移动规则
        piece_name = piece['name']

        if piece_name in ["帅", "将"]:
            return self.is_valid_general_move(from_row, from_col, to_row, to_col)
        elif piece_name in ["仕", "士"]:
            return self.is_valid_advisor_move(from_row, from_col, to_row, to_col)
        elif piece_name in ["相", "象"]:
            return self.is_valid_elephant_move(from_row, from_col, to_row, to_col)
        elif piece_name in ["车"]:
            return self.is_valid_chariot_move(from_row, from_col, to_row, to_col)
        elif piece_name in ["马"]:
            return self.is_valid_horse_move(from_row, from_col, to_row, to_col)
        elif piece_name in ["炮"]:
            return self.is_valid_cannon_move(from_row, from_col, to_row, to_col)
        elif piece_name in ["兵", "卒"]:
            return self.is_valid_pawn_move(from_row, from_col, to_row, to_col)

        return False

    def is_valid_general_move(self, from_row, from_col, to_row, to_col):
        """检查将/帅的移动是否合法"""
        piece = self.board[from_row][from_col]
        color = piece['color']

        # 判断是否在九宫格内
        if color == "red":
            # 红方九宫格范围：(3-5, 7-9)
            if not (3 <= to_col <= 5 and 7 <= to_row <= 9):
                return False
        else:
            # 黑方九宫格范围：(3-5, 0-2)
            if not (3 <= to_col <= 5 and 0 <= to_row <= 2):
                return False

        # 走一步（上下左右）
        dx = abs(to_col - from_col)
        dy = abs(to_row - from_row)
        return (dx == 1 and dy == 0) or (dx == 0 and dy == 1)

    def is_valid_advisor_move(self, from_row, from_col, to_row, to_col):
        """检查仕/士的移动是否合法"""
        piece = self.board[from_row][from_col]
        color = piece['color']

        # 判断是否在九宫格内
        if color == "red":
            # 红方九宫格范围：(3-5, 7-9)
            if not (3 <= to_col <= 5 and 7 <= to_row <= 9):
                return False
        else:
            # 黑方九宫格范围：(3-5, 0-2)
            if not (3 <= to_col <= 5 and 0 <= to_row <= 2):
                return False

        # 走斜线一步
        dx = abs(to_col - from_col)
        dy = abs(to_row - from_row)
        return dx == 1 and dy == 1

    def is_valid_elephant_move(self, from_row, from_col, to_row, to_col):
        """检查相/象的移动是否合法"""
        piece = self.board[from_row][from_col]
        color = piece['color']

        # 不能过河
        if color == "red" and to_row < 5:
            return False
        if color == "black" and to_row > 4:
            return False

        # 走田字
        dx = abs(to_col - from_col)
        dy = abs(to_row - from_row)
        if dx != 2 or dy != 2:
            return False

        # 检查象眼
        middle_row = (from_row + to_row) // 2
        middle_col = (from_col + to_col) // 2
        if self.board[middle_row][middle_col] is not None:
            return False

        return True

    def is_valid_chariot_move(self, from_row, from_col, to_row, to_col):
        """检查车的移动是否合法"""
        # 直线移动
        if from_row != to_row and from_col != to_col:
            return False

        # 检查路径上是否有其他棋子
        if from_row == to_row:  # 横向移动
            start = min(from_col, to_col) + 1
            end = max(from_col, to_col)
            for col in range(start, end):
                if self.board[from_row][col] is not None:
                    return False
        else:  # 纵向移动
            start = min(from_row, to_row) + 1
            end = max(from_row, to_row)
            for row in range(start, end):
                if self.board[row][from_col] is not None:
                    return False

        return True

    def is_valid_horse_move(self, from_row, from_col, to_row, to_col):
        """检查马的移动是否合法"""
        dx = abs(to_col - from_col)
        dy = abs(to_row - from_row)

        # 马走日
        if not ((dx == 1 and dy == 2) or (dx == 2 and dy == 1)):
            return False

        # 检查蹩马腿
        if dx == 1:  # 竖日
            leg_row = from_row + (1 if to_row > from_row else -1)
            if self.board[leg_row][from_col] is not None:
                return False
        else:  # 横日
            leg_col = from_col + (1 if to_col > from_col else -1)
            if self.board[from_row][leg_col] is not None:
                return False

        return True

    def is_valid_cannon_move(self, from_row, from_col, to_row, to_col):
        """检查炮的移动是否合法"""
        # 直线移动
        if from_row != to_row and from_col != to_col:
            return False

        # 目标位置的棋子
        target = self.board[to_row][to_col]

        # 计算路径上的棋子数量
        pieces_in_path = 0
        if from_row == to_row:  # 横向移动
            start = min(from_col, to_col) + 1
            end = max(from_col, to_col)
            for col in range(start, end):
                if self.board[from_row][col] is not None:
                    pieces_in_path += 1
        else:  # 纵向移动
            start = min(from_row, to_row) + 1
            end = max(from_row, to_row)
            for row in range(start, end):
                if self.board[row][from_col] is not None:
                    pieces_in_path += 1

        # 吃子时必须隔一个棋子
        if target is not None:
            return pieces_in_path == 1
        # 移动时中间不能有棋子
        else:
            return pieces_in_path == 0

    def is_valid_pawn_move(self, from_row, from_col, to_row, to_col):
        """检查兵/卒的移动是否合法"""
        piece = self.board[from_row][from_col]
        color = piece['color']

        # 只能走一步
        dx = abs(to_col - from_col)
        dy = abs(to_row - from_row)
        if dx + dy != 1:
            return False

        # 红方兵
        if color == "red":
            # 没过河只能前进
            if from_row > 4:  # 在己方区域
                return to_row == from_row - 1 and to_col == from_col
            # 过河后可以左右移动
            else:
                if dx == 1:  # 左右移动
                    return to_row == from_row
                else:  # 上下移动
                    return to_row == from_row - 1  # 只能前进

        # 黑方卒
        else:
            # 没过河只能前进
            if from_row < 5:  # 在己方区域
                return to_row == from_row + 1 and to_col == from_col
            # 过河后可以左右移动
            else:
                if dx == 1:  # 左右移动
                    return to_row == from_row
                else:  # 上下移动
                    return to_row == from_row + 1  # 只能前进

    def restart_game(self):
        """重新开始游戏"""
        # 清除棋盘上的所有棋子
        for row in range(self.BOARD_HEIGHT):
            for col in range(self.BOARD_WIDTH):
                piece = self.board[row][col]
                if piece is not None:
                    self.canvas.delete(piece['circle'])
                    self.canvas.delete(piece['text'])

        # 重置棋盘数据
        self.board = [[None for _ in range(self.BOARD_WIDTH)] for _ in range(self.BOARD_HEIGHT)]

        # 重置游戏状态
        self.selected_piece = None
        self.current_player = 'red'
        self.game_over = False
        self.status_var.set("红方回合")

        # 重新初始化棋子
        self.init_pieces()

if __name__ == "__main__":
    root = tk.Tk()
    game = ChineseChess(root)
    root.mainloop()